# TODO:
- generally move everything to other places and delete this notebook
- Ax parameter optimization -> predictor_train.ipynb
- The stuff about filtering the data move to train_eval (trial_to_samples). Use two kalmans for each coord and add option to filter data before making sequences.

In [2]:
import os
os.chdir('/app/Pogona_realtime/Arena')

In [3]:
import imp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from time import time
from ax.service.managed_loop import optimize
import torch
import json
import cv2 as cv

In [4]:
from Prediction import dataset
from Prediction import kalman_predict
from Prediction import visualize
from Prediction import train_eval

In [ ]:
imp.reload(dataset)
imp.reload(visualize)
imp.reload(train_eval)

In [5]:
all_df = dataset.collect_data(data_sources={'detections': True, 'timestamps': True, 'dlc': False, 'touches': True})

110 trials loaded


In [6]:
trial_names = list(all_df.index.unique())

In [22]:
trial = trial_names[8]
homography = dataset.homography_for_trial(trial)
correction_fn = visualize.get_correction_fn(homography,1920)
vid_path = dataset.get_trial_video_path(trial)
bboxes = all_df.loc[trial][['x1', 'y1', 'x2', 'y2']].values

In [55]:
# useful for filtering the dataframe.
def filter_bboxes(bboxes, R, q_var):
    traj_inited = False
    filtered_bboxes = np.empty(bboxes.shape)
    filtered_bboxes[:] = np.nan

    for i in range(bboxes.shape[0]):
        if not np.any(np.isnan(bboxes[i])):
            if traj_inited:
                kf.update(bboxes[i])
            else:
                traj_inited = True
                init_x = np.zeros(8, np.float)
                init_x[0::2] = bboxes[i]
                kf = kalman_predict.create_kalman_filter(input_dim=4, init_x=init_x, r_var=50.0, q_var=q_var)
                kf.R = R
    
        if traj_inited:
            kf.predict()
            filtered_bboxes[i] = kf.x[0::2]
    
    return filtered_bboxes

## Search for q_var and r_var for prediction

In [7]:
parameters=[
    {"name": "q_var", "type": "range", "bounds": [0.0, 100.0]},
    {"name": "r_var", "type": "range", "bounds": [0.0, 1000.0]},
    ]

In [8]:
num_derivs = 2

def eval_kalman_predictor(params):
    total_ADE = 0
    start_time = time()
    
    for trial in trial_names:        
        trial_bboxes = all_df.loc[trial][['x1', 'y1', 'x2', 'y2']].values        
        predictor = kalman_predict.KalmanPredictor(20, num_derivs, params["q_var"], params["r_var"])
        results, forecasts = train_eval.eval_trajectory_predictor(predictor, trial_bboxes, show_progress=False)
        total_ADE += results["avg ADE"]

        print(".", end="")
    mean_ADE = total_ADE / len(trial_names)
    print(f"params: {params} ADE:{mean_ADE} duration: {time() - start_time:.2f}")
    
    return mean_ADE

In [9]:
best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=eval_kalman_predictor,
    objective_name='ADE',
    total_trials=100, 
    minimize=True)

[INFO 10-18 18:13:21] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.
[INFO 10-18 18:13:21] ax.service.managed_loop: Started full optimization with 100 steps.
[INFO 10-18 18:13:21] ax.service.managed_loop: Running optimization trial 1...


.............................................................................................................

[INFO 10-18 18:15:24] ax.service.managed_loop: Running optimization trial 2...


.params: {'q_var': 67.19647450372577, 'r_var': 907.0090558379889} ADE:83.6922897400524 duration: 122.83
.............................................................................................................

[INFO 10-18 18:17:27] ax.service.managed_loop: Running optimization trial 3...


...

.params: {'q_var': 99.99999999999996, 'r_var': 367.59331896053004} ADE:115.38675116333704 duration: 124.25
.............................................................................................................

[INFO 10-18 21:36:50] ax.service.managed_loop: Running optimization trial 99...


.params: {'q_var': 99.99999999999997, 'r_var': 288.9683938766629} ADE:123.11448057464996 duration: 125.78
.............................................................................................................

[INFO 10-18 21:38:56] ax.service.managed_loop: Running optimization trial 100...


.params: {'q_var': 11.706953182482163, 'r_var': 539.0032665742581} ADE:64.83567163154748 duration: 124.33
..............................................................................................................params: {'q_var': 0.0, 'r_var': 111.37135445725701} ADE:159.40399677426805 duration: 125.22


### Best parameters for constant acceleration model

- {'q_var': 5.232464612645584, 'r_var': 76.11997188671926} ADE:77.80251293150333
- {'q_var': 8.305751509936852e-11, 'r_var': 6.104439686140315} ADE:56.59084980428664
- {'q_var': 11.243166495979125, 'r_var': 100.0} ADE:87.78338522412209
- {'q_var': 6.60728768670246, 'r_var': 68.14068332725518} ADE:84.5807759587622
- {'q_var': 3.179927344778319e-08, 'r_var': 99.99999991206423} ADE:47.3384215434221
- {'q_var': 1.0454159059256132, 'r_var': 250.94883769233357} ADE:49.15010262136672 (b
- {'q_var': 4.584311371533344, 'r_var': 99.99999999999997} ADE:70.82873570482703

### Best parameters for constant velocity model

- {'q_var': 5.370325791258682, 'r_var': 120.38665794106605} ADE:20.309112430927975
- {'q_var': 17.93990045743253, 'r_var': 134.50567199523354} ADE:22.291919662508487
- {'q_var': 31.548048307438687, 'r_var': 199.99999999999997} ADE:22.69878701033521
- {'q_var': 17.266130167990923, 'r_var': 162.3100133612752} ADE:21.791229502949385
- {'q_var': 2.1451830012770112, 'r_var': 119.49832681948244} ADE:19.396253677900415
- {'q_var': 10.68977698973989, 'r_var': 104.58652082983697} ADE:21.706828026167155
- {'q_var': 20.018173460688566, 'r_var': 180.35298309143698} ADE:21.881616195822115

# What to say about Kalman
- uses two kalman filters so we can use diagonal matrices (assume independence between x and y coordinate but not between x1 and x2 or y1 and y2).
- hyper-parameter search using AX again for choosing the best measurement noise variance and process noise variance values.
    - there might be better ways but its difficult to measure these values (or requires more knowledge of the subject).
- values are not bad when using constant velocity model. Constant acceleration is much worse.
- now seems to achieve better ADE than GRUEncDec??
- would like to test constant accel again with higher R values (last time was up to 100 i think)
- what about suggesting first filtering with kalman and then making predictions with RNN?
- maybe we should compare the results for short horizons to see if it's worthwhile to filter the data before using RNN